In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

In [21]:
import os
from PIL import Image
import pickle

# define the path to the main folder
main_folder = "skin-cancers"

# define the path to the test folder
test_folder = os.path.join(main_folder, "Test")

# define the path to the train folder
train_folder = os.path.join(main_folder, "Train")

# create empty lists to store the test and train images
test_images = []
train_images = []

# loop through the test folders and open the image files
for label_folder in os.listdir(test_folder):
    label_path = os.path.join(test_folder, label_folder)
    for image_file in os.listdir(label_path):
        image = Image.open(os.path.join(label_path, image_file))
        test_images.append(image)

# loop through the train folders and open the image files
for label_folder in os.listdir(train_folder):
    label_path = os.path.join(train_folder, label_folder)
    for image_file in os.listdir(label_path):
        image = Image.open(os.path.join(label_path, image_file))
        train_images.append(image)

# save the test and train images as pickle files
with open("test.p", "wb") as f:
    pickle.dump(test_images, f)
with open("train.p", "wb") as f:
    pickle.dump(train_images, f)